In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchFrameException
import re

In [3]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['cafe_name', "cafe_type", 'starring', 'visitor_review', 'blog_review', 'address', 'business_hours', 'phone_number', 'connection_site', 'description',
"menu", "keyword", 'keyword_number', 'image_link'])

In [3]:
cafe_name_list = ['우주라이크커피 경산점',
 '하늘아래사진관',
 '말그미옥상',
 '호미화방',
 '코너커피',
 '줄라이모닝',
 '카페로커피',
 '엠프티커피',
 '다올',
 '텀블루']

In [59]:
def cafe_crawling(driver):
#     try:
#         WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "place_section no_margin OP4V8")))
#         time.sleep(1)
#     except:
#         driver = switch_driver(driver, "searchIframe")
#         print(1)
#         return None
    time.sleep(1.5)
    try:
        description_a_link = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div/div/div[7]/div/a/span[1]')
        description_a_link.click()
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    reviews_div = soup.find('div', class_='dAsGb')
    starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    
    if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
        return None
    
    visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    if visitor_reviews < 10:
        return None
    
    description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else "" 
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
    keyword_list_link.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(soup.find('em').text)

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number]

In [60]:
def switch_driver(driver, iframe):
    try:
        driver.switch_to.default_content()
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, iframe)))
    except NoSuchFrameException:
        print(f"Frame '{iframe}' not found.")
    return driver

In [71]:
# Selenium을 사용하여 웹 드라이버 초기화
driver = webdriver.Chrome()

#... 이전 코드 ...

for cafe_name in cafe_name_list:
    # 검색
    driver.get("https://map.naver.com/p/search/" + cafe_name)
    time.sleep(1.5)  # 추가 대기 시간
    
    current_url = driver.current_url
    url_point = current_url.split("/")

    if len(url_point) <= 6:
        driver = switch_driver(driver, "searchIframe")
        scrollable_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))

        for _ in range(5):
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
            time.sleep(1)  # 적절한 로딩 시간 기다림

        li_elements = scrollable_div.find_elements(By.TAG_NAME, "li") 

        for idx in range(len(li_elements)):
            try:
                # 페이지나 프레임 전환 후 요소를 다시 찾습니다.
                scrollable_div = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
                cafe_link = scrollable_div.find_element(By.XPATH, f'./ul/li[{idx+1}]/div[1]/div[2]/a[1]')
                cafe_link.click()

                driver = switch_driver(driver, "entryIframe")

                cafe_row = cafe_crawling(driver)
                if cafe_row:
                    df.loc[len(df)] = cafe_row
                    print(cafe_row)

                driver = switch_driver(driver, "searchIframe")
            except StaleElementReferenceException:
                # 요소가 더 이상 유효하지 않을 경우 다시 시도
                continue
    else:   
        driver = switch_driver(driver, "entryIframe")
        cafe_row = cafe_crawling(driver)
        if cafe_row:
            df.loc[len(df)] = cafe_row
            print(cafe_row)


['우주라이크커피 경산점', '카페', 4.77, 84, 4, '경북 경산시 대학로10길 24 1층', '21:00에 영업 종료', '053-710-8852', '', '반갑습니다 : ) 우주라이크 커피 경산점 인사드립\n니다! 새롭고, 신선하고, 합리적인 커피! 브라질 내\n추럴, 콜롬비아 수프리모 워시드, 에티오피...', '', {'커피가 맛있어요': 34, '친절해요': 23, '디저트가 맛있어요': 18, '음료가 맛있어요': 15, '가성비가 좋아요': 12, '특별한 메뉴가 있어요': 9, '매장이 청결해요': 9, '대화하기 좋아요': 7, '주차하기 편해요': 6, '좌석이 편해요': 5}, 61]
['말그미옥상', '케이크전문', 4.55, 91, 153, '대구 남구 계명중앙1길 25 2층', '20:00에 영업 종료', '0507-1310-3966', '', "'맑음' 이라는 말에서 따온 순 우리말\n말그미라는 단어를 사용한 '말그미옥상' 입니다.", '', {'커피가 맛있어요': 35, '디저트가 맛있어요': 17, '대화하기 좋아요': 17, '음료가 맛있어요': 16, '친절해요': 15, '인테리어가 멋져요': 10, '매장이 청결해요': 10, '뷰가 좋아요': 10, '특별한 메뉴가 있어요': 6, '사진이 잘 나와요': 4}, 69]
Stale element reference. Attempting to find element again.
Timeout while waiting for element.
Stale element reference. Attempting to find element again.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout w